# Activities

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

The activities performed are:

- Discuss the fact that we now want to be able to allow access to our data from other locations, which is the overall goal of a distributed application
- In order to be able to access data, we create an HTTP API (short API from now on)
- An API is a series of methods that can be implicitly called when some URL's are visited
    - A URL request also contains a request body
    - Method parameters are passed as elements of the URL, or inside the body
    - A URL request also contains a tag (_HTTP method_)
        - `GET` 
        - `POST`
        - `PUT`
        - `DELETE`
    - A router, or routing rules, define which method is called by which URL, and what parts of the URL (if any) are parsed as parameters of the method
- All the API (methods) related to a single entity (for example `Student`) are usually put in a single class (in our example below, StudentApiController), but YMMV (_your mileage may vary_)
- What methods do we add?
    - A very common approach to structure data access is called REST
        - REST means _representational state transfer_
    - REST assumes that all calls related to an entity are found based on a single URL, such as `xxx.com/students`
    - the most common methods are
        - `GET` on `xxx.com/students` will give us back all students
        - `GET` on `xxx.com/students/{id}` will give us back the student with `id = {id}`
        - `POST` on `xxx.com/students` will create a new student (usually the student to add is in the body of the request)
        - `PUT` on `xxx.com/students/{id}` will change student with `id = {id}` to the value sent through the body
        - `DELETE` on `xxx.com/students/{id}` will remove the student with `id = {id}`
    - methods do not always succeed right away, so some methods need to exhibit some additional properties if the network layer sends them multiple times in a row
    - `GET` must be nullipotent: it must not change the database
    - `PUT/POST/DELETE` must be idempotent: the same call, multiple times in a row must produce the same result
- Example of such a class in C#:

```
[Route("api/students")]
public class StudentApiController : Controller
{
  [HttpGet]
  public IActionResult GetAll()
  {
    ...
    return Ok(...);
  }

  [HttpGet("{id}")]
  public IActionResult GetById(int id)
  {
    ...
    return Ok(...);
  }  

  [HttpPost]
  public IActionResult Create([FromBody] Student student)
  {
    ...
    return Ok(...);
  }

  [HttpPut]
  public IActionResult Update([FromBody] Student modified_student)
  {
    ...
    return Ok(...);
  }

  [HttpDelete("{id}")]
  public IActionResult Delete(int id)
  {
    ...
    return Ok(...);
  }
}

```

- The class above implements a RESTful interface on `xxx/students`. In general, in .NET an application is RESTful if:
    - there is a class implementing the router attribute (`[Route("...")]`)
    - there is a series of methods implementing get, post, put and delete operations
    - `startup.cs` contains `app.UseMvc(routes => {});` (null configuration), necessary to register the router to the services exposed by the server
- An API call automatically transfers data via JSON, which is serialized and deserialized automatically (structural errors prevent completion of the call for type safety and security reasons)
- The body of the methods is just a combination of LINQ and regular code
    - `Ok` denotes that we give a result back (the argument is the result, seralized to JSON)
    - There are various methods such as `Ok` that we can give to signal errors, such as _not found_, _unauthorized_, and much more
- Suppose now that we have a lot of students (hundreds of thousands)
    - `GetAll` is a bit of an issue
    - A malicious attacker could cause a DDOS (distributed denial of service) by simply spinning up a lot of requests
    - Requests would all eventually fail because of timeout
    - Our server would be quickly overwhelmed
- The solution is pagination: we return a block of data (called _a page_), never bigger than a given amount of elements

```
[HttpGet]
public IActionResult GetAll([FromQuery] int page_index, [FromQuery] int page_size)
```

- `GetAll` will now return the elements from `page_index * page_size` to `page_index * page_size + page_size - 1`
- We can impose a maximum cap for `page_size`, so that if the user asks for more than, say, 100 students, we still only give 100 back
- `GetAll` will therefore return a JSON result looking like

```
{
    page_index: ...
    page_size: ...
    num_pages: ...
    students: [ ... ]    
}
```

- notice that the parameters `page_index` and `page_size` come from the query part of the URL itself
    - For example: `xxx/students?page_index=10&page_size=25`
    - Everything after the `?` is the query

- `GetAll` now contains basic filtering (basic because it is simply a range)
- it is possible to add more parameters to perform extra filtering
- a typical strategy is also to define a parameterized, hierarchical data structure which specifies a filter

```
Expr = And(Expr, Expr) | Or(Expr,Expr) | GreaterThan(Property,Const) | ...
```
    
- the `GetAll` method can then parse and transform this data structure into a `Where` clause dynamically
    - LINQ also offers `Expr`, which is an interface to automatically build predicates and other expressions dynamically
- `GetAll` can also receive as optional parameters one or more properties to sort by (`OrderBy`)

- JSON is not efficient compared to binary, but it is helpful for humans
    - Compare the size of `{ Name:"Johnatan", Surname:"Ridderkerk", BirthDate:02/03/1996 }` in JSON (count the characters: $63$ bytes, even more in UTF16) to the binary size: $\mathbb{S}(8) \times \mathbb{S}(10) \times \mathbb{I} \times \mathbb{I} \times \mathbb{I} = 9 + 11 + 4 + 4 + 4 = 32$ bytes
    - Be careful, because the binary serializer can make a huge performance difference